In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from folium.plugins import HeatMap
import folium

from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

In [24]:
import pandas as pd
data=pd.read_csv('../merged_data.csv.zip')
data = data.drop(data.columns[21:], axis=1) #dropped here because no data in these columns and easier to do here than in dataset
data

In [22]:
data.shape

(488457, 21)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 488457 entries, 1 to 550579
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    488457 non-null  int64  
 1   YEAR_MONTH                    488457 non-null  int64  
 2    COMMISSIONER_CODE            488457 non-null  object 
 3    COMMISSIONER_NAME            488457 non-null  object 
 4    CONTRACT_NUMBER              488457 non-null  float64
 5    PROVIDER_NAME                488457 non-null  object 
 6    LATEST_PPC_ADDRESS_POSTCODE  488457 non-null  object 
 7    LSOA11_CODE                  488457 non-null  object 
 8    UDA_PERF_TARGET              488457 non-null  float64
 9    UDA_FIN_VAL                  488457 non-null  float64
 10   UDA_DELIVERED                488457 non-null  float64
 11   LATE_SUBMITTED_FP17          488457 non-null  float64
 12   UDA_DELIVERED_FD             488457 non-null  fl

DATA CLEANUP
- duplicates; null values; extra columns; commas/capitals
- categorical columns to numerical
- uniform dates
- postcodes
- scale 

In [ ]:
data.duplicated()

In [21]:
data.drop_duplicates()

KeyboardInterrupt: 

In [20]:
nan_counts = data.isna().sum()
nan_counts

Unnamed: 0                      0
YEAR_MONTH                      0
 COMMISSIONER_CODE              0
 COMMISSIONER_NAME              0
 CONTRACT_NUMBER                0
 PROVIDER_NAME                  0
 LATEST_PPC_ADDRESS_POSTCODE    0
 LSOA11_CODE                    0
 UDA_PERF_TARGET                0
 UDA_FIN_VAL                    0
 UDA_DELIVERED                  0
 LATE_SUBMITTED_FP17            0
 UDA_DELIVERED_FD               0
 BAND_1_DELIVERED               0
 BAND_2_DELIVERED               0
 BAND_3_DELIVERED               0
 BAND_URGENT_DELIVERED          0
 BAND_OTHER_DELIVERED           0
 CHILD_12M_COUNT                0
 ADULT_24M_COUNT                0
 GENERAL_DENTAL_FIN_VALUE       0
dtype: int64

In [19]:
data[data.isna().any(axis=1)].head()


,Unnamed: 0,YEAR_MONTH,COMMISSIONER_CODE,COMMISSIONER_NAME,CONTRACT_NUMBER,PROVIDER_NAME,LATEST_PPC_ADDRESS_POSTCODE,LSOA11_CODE,UDA_PERF_TARGET,UDA_FIN_VAL,UDA_DELIVERED,LATE_SUBMITTED_FP17,UDA_DELIVERED_FD,BAND_1_DELIVERED,BAND_2_DELIVERED,BAND_3_DELIVERED,BAND_URGENT_DELIVERED,BAND_OTHER_DELIVERED,CHILD_12M_COUNT,ADULT_24M_COUNT,GENERAL_DENTAL_FIN_VALUE


In [18]:
data = data.ffill() #choosing not to drop the missing values but to forward fill in order to build a monthly time series 

In [17]:
data.isnull().sum()

Unnamed: 0                      0
YEAR_MONTH                      0
 COMMISSIONER_CODE              0
 COMMISSIONER_NAME              0
 CONTRACT_NUMBER                0
 PROVIDER_NAME                  0
 LATEST_PPC_ADDRESS_POSTCODE    0
 LSOA11_CODE                    0
 UDA_PERF_TARGET                0
 UDA_FIN_VAL                    0
 UDA_DELIVERED                  0
 LATE_SUBMITTED_FP17            0
 UDA_DELIVERED_FD               0
 BAND_1_DELIVERED               0
 BAND_2_DELIVERED               0
 BAND_3_DELIVERED               0
 BAND_URGENT_DELIVERED          0
 BAND_OTHER_DELIVERED           0
 CHILD_12M_COUNT                0
 ADULT_24M_COUNT                0
 GENERAL_DENTAL_FIN_VALUE       0
dtype: int64

In [16]:
data = data.dropna()

In [9]:
data.columns

Index(['Unnamed: 0', 'YEAR_MONTH', ' COMMISSIONER_CODE', ' COMMISSIONER_NAME',
       ' CONTRACT_NUMBER', ' PROVIDER_NAME', ' LATEST_PPC_ADDRESS_POSTCODE',
       ' LSOA11_CODE', ' UDA_PERF_TARGET', ' UDA_FIN_VAL', ' UDA_DELIVERED',
       ' LATE_SUBMITTED_FP17', ' UDA_DELIVERED_FD', ' BAND_1_DELIVERED',
       ' BAND_2_DELIVERED', ' BAND_3_DELIVERED', ' BAND_URGENT_DELIVERED',
       ' BAND_OTHER_DELIVERED', ' CHILD_12M_COUNT', ' ADULT_24M_COUNT',
       ' GENERAL_DENTAL_FIN_VALUE'],
      dtype='object')

In [15]:
data = data.drop(['CONTRACT_NUMBER', 'LSOA11_CODE', 'COMMISSIONER_CODE'], axis=1)
data

KeyError: "['CONTRACT_NUMBER', 'LSOA11_CODE', 'COMMISSIONER_CODE'] not found in axis"

In [ ]:
# Create LSOA11 mappings for each nation
conditions = [
    df['LSOA11_CODE'].str.startswith('E'),
    df['LSOA11_CODE'].str.startswith('W'),
    df['LSOA11_CODE'].str.startswith('S')
]

values = ['England', 'Wales', 'Scotland']

# Create the new column "nation"
df['NATION'] = np.select(conditions, values, default='Unknown')

make uniform the date:

In [ ]:
df['Newcol'] = pd.to_datetime(df['Mycol'], format='%Y%M')
pd.to_datetime(df['Mycol'], format='%Y%M')